Many search nerds get an instinct they want to "learn the right boosts" to apply to their queries. Search often feels like whack-a-mole, and often folks say "if I could just optimize the boost on say the 'title match' vs the boost on the 'body match' I'd be in great shape!"

This instinct of learning what weight to apply to a single query you're summing with a bunch of other queries is the instinct behind the simplest [learning to rank](http://opensourceconnections.com/blog/2017/02/24/what-is-learning-to-rank/) model: the linear model. Yes! Good ole linear regression! What's nice about linear regression is that, well, it doesn't really *feel* like machine learning. It feels like high school statistics! It's very easy to understand the model and make sense of it.

In this series of articles, I want to introduce you to begin to introduce the key algorithms behind successful learning to rank implementations, starting with linear regression and working up to topics like gradient boosting and random forests.

## Learning to Rank as a Regression Problem
For this series of articles, I want to map learning to rank, as you might be familiar from [previous articles](http://opensourceconnections.com/blog/2017/02/24/what-is-learning-to-rank/) and [documentation](https://github.com/o19s/elasticsearch-learning-to-rank#building-a-learning-to-rank-system-with-elasticsearch) to a more general problem: regression. *Regression* trains a model to map a set of numerical features to a predicted numerical value. 

For example, what if you wanted to be able to predict a company's profit? You might have, on hand, historical data about public corporations including number of employees, stock market price, revenue, cash on hand, etc. Given data you know about existing companies, your model could be trained to predict profit as a function of these variables (or a subset thereof). For a new company you could use your function to arrive at a prediction of the company's profit.

Just the same, learning to rank can be a regression problem. You have on hand a series of *judgments* that grade how relevant a document is for a query. Our relevance grades could range from A to F. More commonly they range from 0 (not at all relevant) to 4 (exactly relevant). If we just consider a keyword search to be a query, this become, as an example:


    grade,movie,keywordquery
    4,Rocky,rocky
    0,Turner and Hootch,rocky
    3,Rocky II,rocky
    1,Rambo,rocky
    ...
    
    
Learning to Rank becomes a regression problem when you build a model to predict the *grade* as a function of ranking-time *signals.* Recall from [Relevant Search](http://manning.com/books/relevant-search) we term signals to mean any measurement about the relationship between the query and a document. Often signals are *query-dependent* - that is they result by taking some measurement of how a keyword (or other part of the query) relates to the document. Other times they are query or document-only, such as publication date, or whether a "company name" could be extracted from the query using NLP method.

In other words, let's consider the movie example above. You might have 2 query-dependent signals you suspect could help predict relevance:

1. How many times a search keyword occurs in the **title** field
2. How many times a search keyword occurs in the **overview** field

Augmenting the *judgments* above, you might arrive at a training set for regression like below:

    ```
    4,1,1
    0,0,0
    3,0,3
    1,0,1
    ```
    
You can apply a regression process such as linear regression, to predict the first column using the other columns. You can build such a system on top of an existing search engine like [Solr](https://cwiki.apache.org/confluence/display/solr/Learning+To+Rank) or [Elasticsearch](http://opensourceconnections.com/blog/2017/02/14/elasticsearch-learning-to-rank/).

Learning to Rank comes with a few extra complications we'll save for a future article

1. Examples are provided in groups, grouped by queries. A document can be a grade 4 for query "rambo" and a 0 for query "beauty and the beast."
2. How do you arrive at these "grades"? 
3. How does all this work in practice?

Yes yes, but for now I want to look at a specific *kind* of regression. It's the magic under learning to rank and I want to teach you all about it. Keep in  mind though that the questions above are often the hard problems. Coming up with supposed data to create a model is fine and dandy, but garbage in, garbage out! 


## What *kind* of regression works best for Learning to Rank?

If you've learned any statistics, you're probably familiar with Linear Regression. *Linear Regression* defines the regression problem as a simple linear function. For example, if in learning to rank we called the first signal above (how many times a search keyword occurs it the **title** field) as `t` and the second signal above (the same for the **overview** field) as `o`, our model might be able to generate a function `s` to score our relevance as follows:

    s(t, o) = c0 + c1 * t + c2 * o
    

We can estimate the best fit coefficients `c0, c1, c2...` that predict our training data using a procedure known as [least squares fitting](http://mathworld.wolfram.com/LeastSquaresFitting.html). We won't cover that here, but the gist is we can find the `c0, c1, c2, ...` that minimize the error between the actual grade, `g` and the prediction `s(t,o)`. It's often as simple as a little [matrix math](https://en.wikipedia.org/wiki/Ordinary_least_squares) if you want to bone up on your linear algebra.

You can get fancier with linear regression, including deciding there's really a third ranking signal, `a` which we can define as `t*o`. Or another signal called t2, which could be in reality `t^2` or `log(t)` or whatever formulation you suspect could help best predict relevance. You can then just treat these values as additional columns in the dataset for which linear regression can learn coefficients for.

There's a deeper art to of designing, testing ,and evaluating models of any flavor, I heartily recommend [Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/) if you'd like to learn more.


## Learning to Rank with Linear Regression in sklearn

To give you a taste, Python's [sklearn](http://scikit-learn.org/stable/) family of libraries is a convenient way to play with regression. If we want to try out the simple learning to rank training set above for linear regression, we can express the relevance grade's we're trying to predict as `S`, and the signals we feel will predict that score as `X`. 

We're going to have some fun with some movie relevance data. Here we have a set of relevance grades for a keyword search "Rocky." Recall above we had a judgment list that we transformed into a training set. Let's take a gander at a real training set (w/ comments to help us see what's going on). The three ranking signals we'll be examining include the title TF\*IDF score, the overview TF\*IDF score and the movie's user rating.

```
grade,titleScore,overviewScore,ratingScore,comment:# <docid> kewyords@movietitle
4,10.65,8.41,7.40,# 1366   rocky@Rocky
3,0.00,6.75,7.00,# 12412  rocky@Creed
3,8.22,9.72,6.60,# 1246   rocky@Rocky Balboa
3,8.22,8.41,0.00,# 1374   rocky@Rocky IV
3,8.22,7.68,6.90,# 1367   rocky@Rocky II
3,8.22,7.15,0.00,# 1375   rocky@Rocky V
3,8.22,5.28,0.00,# 1371   rocky@Rocky III
2,0.00,0.00,7.60,# 154019 rocky@Belarmino
2,0.00,0.00,7.10,# 1368   rocky@First Blood
2,0.00,0.00,6.70,# 13258  rocky@Son of Rambow
2,0.00,0.00,0.00,# 70808  rocky@Klitschko
2,0.00,0.00,0.00,# 64807  rocky@Grudge Match
2,0.00,0.00,0.00,# 47059  rocky@Boxing Gym
...
```

So let's get to cranking out the code! Below we've got code for reading an a CSV into a Numpy Array. The array is two dimensional, the first dimension being row, the second being column. You'll see what the funky slicing is doing to the array in the comments below:


In [51]:
from sklearn.linear_model import LinearRegression
from math import sin
import numpy as np
import csv

rockyData = removeCsvHeader(np.genfromtxt('rocky.csv', delimiter=','))[1:] # Remove the CSV header

rockyGrades = rockyData[:,0]   # Slice out column 0, where the grades are
rockySignals = rockyData[:,1:-1]  # Features in columns 1...all but last column (the comment)

Great! We're ready to perform a simple linear regression. What we have here is a classic overdetermined system: way more equations than unknowns! So we need to use ordinary least squares to estimate the relationship between the features `rockySignals` and the grades `rockyGrades`. Easy peasy, this is what numpy's linear regression does:

In [52]:
butIRegress = LinearRegression()
butIRegress.fit(rockySignals, rockyGrades)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

This gives us the coefficients to use on our ranking signals, along with a y-intercept.

In [53]:
butIRegress.coef_

array([ 0.01388369,  0.2518476 , -0.00360426])

In [54]:
butIRegress.intercept_

0.98677646111347528

Great! Relevance Solved! (right?) We can use these to to create a ranking function!

For now, I'm ignoring a bunch of item's we'll need to consider to evaluate how *good* of a fit this model is to the data. For the sake of this blog post, we just want to see generally how these models work. But it's a good idea to not just assume this model is phenomenal fit to the training data, and always reserve some data for testing! Future blog posts will dive into these topics :)

### Using our Model to score queries

With these coefficients, let's create our own ranking function! We're doing this for illustration purposes only, sk-learn's `LinearRegression` comes [predict method](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) which evaluates the model given an input, but it's more fun to make our own:

In [59]:
def relevanceScore(intercept, titleCoef, overviewCoef, ratingCoef, titleScore, overviewScore, movieRating):
    return intercept + (titleCoef * titleScore) + (overviewCoef * overviewScore) + (ratingCoef * movieRating)

Using this function, we can arrive at relevance scores for these two movies as candidates for movies for the search "Rambo"

```
titleScore,overviewScore,movieRating,comment
12.28,9.82,6.40,# 7555	rambo@Rambo
0.00,10.76,7.10,# 1368	rambo@First Blood
```

Let's score Rambo and First Blood to see which is more relevant for the query "Rambo"!


In [60]:
# Score Rambo
relevanceScore(butIRegress.intercept_, butIRegress.coef_[0], butIRegress.coef_[1], butIRegress.coef_[2], 
               titleScore=12.28, overviewScore=9.82, movieRating=6.40)


3.6073443792384232

In [61]:
# Score First Blood
relevanceScore(butIRegress.intercept_, butIRegress.coef_[0], butIRegress.coef_[1], butIRegress.coef_[2], 
               titleScore=0.00, overviewScore=10.76, movieRating=7.10)


3.6710663661037541

Very close! First Blood narrowly beats out Rambo for the win! This makes sense -- while Rambo is an exact match, First Blood was the original Rambo movie! Well we shouldn't really give our model that much credit, it hasn't seen *that many* examples to capture that level of nuance. What *is* interesting though is that the coefficient for an the overview score is higher than the one for the title score. So at least in the examples our model has seen, more keyword mentions in an overview is the most highly correlated to relevance. We're already learning a great deal about how our user's perceive relevance!

Let's try a completely oddball scenario, Rocky and Bullwinkle:

```
titleScore,overviewScore,movieRating,comment
0.00,0.00,4.00 # 17711	rambo@The Adventures of Rocky & Bullwinkle
```


In [63]:
# Score Rocky and Bulwinkle
relevanceScore(butIRegress.intercept_, butIRegress.coef_[0], butIRegress.coef_[1], butIRegress.coef_[2], 
               titleScore=0.00, overviewScore=0.0, movieRating=4.0)

0.97235943687019455

As we would expect, not so hot!

## Limitations to the Linear Model (aka why boosts suck)

It's been fun to put together this model. It was easy to understand, and produced reasonably sane results. But direct linear combinations of features often fall short for relevance applications. They fall short for the same reason [direct additive boosting falls short](http://www.flax.co.uk/blog/2016/08/19/boosts-considered-harmful-adventures-badly-configured-search/). 

Why? Nuance!

In the previous example, we began to see how some extremely relevant movies do indeed have very high title TF\*IDF relevance scores. Yet our model sided with the overview field as more correlated with relevance. The reality of when titles match and when overview matches actually *depends on other things*. 

There's no one global coefficient in search, a lot depends specifically on the use case behind the keyword. In some cases you search by looking for the exact name of something. For these use cases, you want a model where high title scores come first. In other cases, you're performing research. You don't know quite what you're looking for. So other factors come into play: body fields. 

And of course, just having the *keywords* might not be enough to figure this out! One k


# Junk Below

In [62]:

numFeatures = 10
numDatapoints = 1000


# Generate 100 samples with 10 random features
X = np.random.random(numFeatures*numDatapoints).reshape(numDatapoints,numFeatures)
Y = np.arctan(X).sum(axis=1)



# Generate 100 random values



Y = np.random.random(numDatapoints)

In [98]:
# Step 1, perform a regression over the X & Y values

tree = DecisionTreeRegressor()
tree.n_features=10
tree.fit(X, Y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [99]:
# Step 2. Next we make predictions using X, and see how far we are from Y

Ypredicted = tree.predict(X)
np.power(Y - Ypredicted, 2).sum()

1.5491898320864383e-08

In [101]:
# Wait, what, error is near 0! We've perfectly predicted the training data.
# This isn't surprising. But it's likely an example of overfitting. Let's train on all but the 
# first 10 data points and then recalculate the *test error*
Xtrain = X[100:]
Xtest = X[:100]
Ytrain = Y[100:]
Ytest = Y[:100]
tree.fit(Xtrain, Ytrain)
Ypredicted = tree.predict(X)
np.power(Y - Ypredicted, 2).sum()

37.243293215375147

In [58]:
X[1]

array([ 0.09914091,  0.31433702,  0.34429422,  0.55603666,  0.83777067,
        0.38102517,  0.85519233,  0.8633129 ,  0.50484428,  0.76361867])

In [59]:
X[:10]

array([[ 0.5206042 ,  0.45071729,  0.94032958,  0.68446533,  0.70176207,
         0.75528753,  0.72025058,  0.22522763,  0.18514977,  0.84070608],
       [ 0.09914091,  0.31433702,  0.34429422,  0.55603666,  0.83777067,
         0.38102517,  0.85519233,  0.8633129 ,  0.50484428,  0.76361867],
       [ 0.41816377,  0.53497742,  0.83310324,  0.05345249,  0.39873131,
         0.11529311,  0.56652605,  0.04872523,  0.42650035,  0.41389597],
       [ 0.32663508,  0.30300935,  0.57256465,  0.45474402,  0.37723288,
         0.63898176,  0.25567442,  0.14961114,  0.73494424,  0.29039654],
       [ 0.27869052,  0.07218927,  0.96985285,  0.65412681,  0.10789258,
         0.10191945,  0.4291287 ,  0.61700272,  0.64590024,  0.07351153],
       [ 0.48881607,  0.37273356,  0.16319504,  0.83005925,  0.67285752,
         0.0910396 ,  0.42477796,  0.70907164,  0.04713346,  0.86128398],
       [ 0.67287971,  0.14537552,  0.68258379,  0.21359162,  0.66374476,
         0.57542011,  0.35296409,  0.22235011